In [1]:
# === User-defined Settings for Topology Optimization ===

# Grid resolution (number of finite elements in x and y directions)
# ⚠️ CNN only supports multiples of 8 for Nx and Ny
Nx = 3                       # Number of elements along x-axis
Ny = 2                       # Number of elements along y-axis

# Material properties
E0 = 1.0                     # Young's modulus of solid material
Emin = 1e-9                  # Young's modulus of void
nu = 0.3                     # Poisson's ratio

# Filter and penalization
rmin = 2.0                   # Radius for density filter
penal = 3.0                  # SIMP penalization factor

# Optimization control
max_iterations = 5         # Number of optimization steps
volfrac = 0.35

# # Sanity checks
# assert ML_framework_to_use in ["jax", "torch"]
assert penal >= 1
assert rmin >= 1

In [2]:
# Imports
import jax
# Enable float64 for better numerical stability (especially in FEA)
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

!git clone https://github.com/SNMS95/AutoDiff_in_TO.git
%cd AutoDiff_in_TO/
from fea_bisec_numpy import setup_fea_problem
from custom_vjps import compute_compliance_ad, bisection_alg_ad

Cloning into 'AutoDiff_in_TO'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 24 (delta 5), reused 14 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 35.74 KiB | 1.19 MiB/s, done.
Resolving deltas: 100% (5/5), done.
/content/AutoDiff_in_TO


In [3]:
problem_data = setup_fea_problem(Nx=Nx, Ny=Ny, rmin=rmin, E0=E0, Emin=Emin, nu=nu)

AD_in_TO.ipynb	fea_bisec_numpy.py  LICENSE	 README.md
custom_vjps.py	jax		    __pycache__


In [ ]:
def optimality_criteria_step(x, ke, args):
  """Heuristic topology optimization, as described in the 88 lines paper."""
  c, dc = autograd.value_and_grad(objective)(x, ke, args)
  dv = autograd.grad(mean_density)(x, args)
  x = optimality_criteria_combine(x, dc, dv, args)
  return c, x

def optimality_criteria_combine(x, dc, dv, args, max_move=0.2, eta=0.5):
  """Fully differentiable version of the optimality criteria."""

  volfrac = args['volfrac']

  def pack(x, dc, dv):
    return np.concatenate([x.ravel(), dc.ravel(), dv.ravel()])

  def unpack(inputs):
    x_flat, dc_flat, dv_flat = np.split(inputs, [x.size, x.size + dc.size])
    return (x_flat.reshape(x.shape),
            dc_flat.reshape(dc.shape),
            dv_flat.reshape(dv.shape))

  def compute_xnew(inputs, lambda_):
    x, dc, dv = unpack(inputs)
    # avoid dividing by zero outside the design region
    dv = np.where(np.ravel(args['mask']) > 0, dv, 1)
    # square root is not defined for negative numbers, which can happen due to
    # small numerical errors in the computed gradients.
    xnew = x * np.maximum(-dc / (lambda_ * dv), 0) ** eta
    lower = np.maximum(0.0, x - max_move)
    upper = np.minimum(1.0, x + max_move)
    # note: autograd does not define gradients for np.clip
    return np.minimum(np.maximum(xnew, lower), upper)

  def f(inputs, lambda_):
    xnew = compute_xnew(inputs, lambda_)
    return volfrac - mean_density(xnew, args)

  # find_root allows us to differentiate through the while loop.
  inputs = pack(x, dc, dv)
  lambda_ = autograd_lib.find_root(f, inputs, lower_bound=1e-9, upper_bound=1e9)
  return compute_xnew(inputs, lambda_)